## Human in the loop

In [1]:
# Installation
! pip install agent-framework openai -q
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [ ]:
# Import all required libraries
import os
import asyncio
import json
from typing import Annotated, List, Dict, Optional
from pydantic import Field
from datetime import datetime
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [ ]:
# Configure API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

print("✅ API Key configured!")


✅ API Key configured!


In [4]:
# Define function tools that DON'T need approval

# Safe functions - Read-only operations
def get_account_balance(
    account_id: Annotated[str, Field(description="The account ID to check")]
) -> str:
    """Get the current account balance (read-only, safe operation)"""
    # Simulate database lookup
    balances = {
        "ACC001": 5000.00,
        "ACC002": 12500.50,
        "ACC003": 750.25
    }
    
    balance = balances.get(account_id, 0.00)
    print(f"✅ Retrieved balance for {account_id}: ${balance}")
    return f"Account {account_id} has a balance of ${balance:.2f}"


def get_transaction_history(
    account_id: Annotated[str, Field(description="The account ID")],
    limit: Annotated[int, Field(description="Number of transactions to retrieve")] = 5
) -> str:
    """Get recent transaction history (read-only, safe operation)"""
    # Simulate transaction history
    transactions = [
        {"date": "2025-12-10", "description": "Online Purchase", "amount": -125.50},
        {"date": "2025-12-09", "description": "Salary Deposit", "amount": 3000.00},
        {"date": "2025-12-08", "description": "Grocery Store", "amount": -85.20},
        {"date": "2025-12-07", "description": "Gas Station", "amount": -45.00},
        {"date": "2025-12-06", "description": "Coffee Shop", "amount": -12.50},
    ]
    
    result = f"Last {limit} transactions for {account_id}:\n"
    for i, txn in enumerate(transactions[:limit], 1):
        result += f"{i}. {txn['date']} - {txn['description']}: ${txn['amount']:.2f}\n"
    
    print(f"✅ Retrieved {limit} transactions for {account_id}")
    return result


print("✅ Safe function tools defined!")


✅ Safe function tools defined!


In [5]:
# Define function tools that REQUIRE human approval

# Critical functions - Modify state, handle money, etc.
def transfer_money(
    from_account: Annotated[str, Field(description="Source account ID")],
    to_account: Annotated[str, Field(description="Destination account ID")],
    amount: Annotated[float, Field(description="Amount to transfer")]
) -> str:
    """
    Transfer money between accounts.
    ⚠️ CRITICAL OPERATION - Requires human approval
    """
    print(f"💸 Executing transfer: ${amount:.2f} from {from_account} to {to_account}")
    
    # Simulate transfer
    transaction_id = f"TXN{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    result = {
        "status": "success",
        "transaction_id": transaction_id,
        "from_account": from_account,
        "to_account": to_account,
        "amount": amount,
        "timestamp": datetime.now().isoformat()
    }
    
    return json.dumps(result, indent=2)


def send_email(
    to_email: Annotated[str, Field(description="Recipient email address")],
    subject: Annotated[str, Field(description="Email subject")],
    body: Annotated[str, Field(description="Email body content")]
) -> str:
    """
    Send an email notification.
    ⚠️ CRITICAL OPERATION - Requires human approval
    """
    print(f"📧 Sending email to {to_email}")
    print(f"   Subject: {subject}")
    
    # Simulate email sending
    result = {
        "status": "sent",
        "to": to_email,
        "subject": subject,
        "sent_at": datetime.now().isoformat()
    }
    
    return json.dumps(result, indent=2)


def delete_account(
    account_id: Annotated[str, Field(description="Account ID to delete")]
) -> str:
    """
    Delete an account permanently.
    ⚠️ CRITICAL OPERATION - Requires human approval
    """
    print(f"🗑️  Deleting account: {account_id}")
    
    # Simulate deletion
    result = {
        "status": "deleted",
        "account_id": account_id,
        "deleted_at": datetime.now().isoformat(),
        "message": f"Account {account_id} has been permanently deleted"
    }
    
    return json.dumps(result, indent=2)


def process_payment(
    amount: Annotated[float, Field(description="Payment amount")],
    card_number: Annotated[str, Field(description="Credit card number (last 4 digits)")],
    description: Annotated[str, Field(description="Payment description")]
) -> str:
    """
    Process a payment transaction.
    ⚠️ CRITICAL OPERATION - Requires human approval
    """
    print(f"💳 Processing payment: ${amount:.2f}")
    
    # Simulate payment processing
    result = {
        "status": "completed",
        "amount": amount,
        "card_last4": card_number[-4:] if len(card_number) >= 4 else "****",
        "description": description,
        "transaction_id": f"PAY{datetime.now().strftime('%Y%m%d%H%M%S')}",
        "timestamp": datetime.now().isoformat()
    }
    
    return json.dumps(result, indent=2)


print("✅ Critical function tools defined (require approval)!")


✅ Critical function tools defined (require approval)!


In [6]:
# Human-in-the-Loop Manager
class HumanInTheLoopManager:
    """
    Manages human approval requests for critical AI agent operations.
    Implements the Human-in-the-Loop (HITL) pattern.
    """
    
    def __init__(self, auto_approve_safe_operations: bool = True):
        self.auto_approve_safe_operations = auto_approve_safe_operations
        self.approval_history: List[Dict] = []
        
        # Define which functions require approval
        self.critical_functions = [
            "transfer_money",
            "send_email",
            "delete_account",
            "process_payment"
        ]
        
        # Safe functions that can auto-approve
        self.safe_functions = [
            "get_account_balance",
            "get_transaction_history"
        ]
    
    def requires_approval(self, function_name: str) -> bool:
        """Check if a function requires human approval"""
        return function_name in self.critical_functions
    
    def display_approval_request(self, function_name: str, parameters: Dict) -> None:
        """Display approval request details to human"""
        print("\n" + "="*70)
        print("🚨 HUMAN APPROVAL REQUIRED")
        print("="*70)
        print(f"Function: {function_name}")
        print(f"Parameters:")
        for key, value in parameters.items():
            print(f"  • {key}: {value}")
        print("="*70)
    
    def get_human_approval(
        self, 
        function_name: str, 
        parameters: Dict,
        reasoning: str = ""
    ) -> tuple[bool, Optional[str]]:
        """
        Request human approval for a function call.
        Returns: (approved: bool, modified_params: Optional[str])
        """
        # Auto-approve safe operations if enabled
        if self.auto_approve_safe_operations and function_name in self.safe_functions:
            print(f"✅ Auto-approved safe operation: {function_name}")
            return True, None
        
        # Display approval request
        self.display_approval_request(function_name, parameters)
        
        if reasoning:
            print(f"AI Reasoning: {reasoning}\n")
        
        # Get human decision
        while True:
            decision = input("Approve this action? (yes/no/modify): ").lower().strip()
            
            if decision in ['yes', 'y']:
                print("✅ Approved by human\n")
                self._log_approval(function_name, parameters, approved=True)
                return True, None
            
            elif decision in ['no', 'n']:
                print("❌ Rejected by human\n")
                self._log_approval(function_name, parameters, approved=False)
                return False, None
            
            elif decision == 'modify':
                print("📝 Modify parameters (enter as JSON):")
                try:
                    modified = input("New parameters: ")
                    modified_params = json.loads(modified)
                    print("✅ Parameters modified\n")
                    self._log_approval(function_name, modified_params, approved=True, modified=True)
                    return True, json.dumps(modified_params)
                except json.JSONDecodeError:
                    print("❌ Invalid JSON. Try again.")
            
            else:
                print("❌ Invalid input. Please enter 'yes', 'no', or 'modify'")
    
    def _log_approval(
        self, 
        function_name: str, 
        parameters: Dict, 
        approved: bool,
        modified: bool = False
    ) -> None:
        """Log approval history"""
        log_entry = {
            "timestamp": datetime.now().isoformat(),
            "function": function_name,
            "parameters": parameters,
            "approved": approved,
            "modified": modified
        }
        self.approval_history.append(log_entry)
    
    def get_approval_history(self) -> List[Dict]:
        """Get the approval history"""
        return self.approval_history
    
    def export_approval_log(self, filename: str = "approval_log.json") -> None:
        """Export approval history to file"""
        with open(filename, 'w') as f:
            json.dump(self.approval_history, f, indent=2)
        print(f"📄 Approval log exported to: {filename}")


print("✅ HumanInTheLoopManager class defined!")


✅ HumanInTheLoopManager class defined!


In [7]:
# Human-in-the-Loop Agent Wrapper
class HITLAgent:
    """
    Wrapper for ChatAgent with Human-in-the-Loop capabilities.
    Intercepts critical function calls and requests human approval.
    """
    
    def __init__(
        self, 
        model: str = "gpt-4o-mini",
        instructions: str = "",
        tools: List = None
    ):
        self.model = model
        self.instructions = instructions
        self.tools = tools or []
        self.hitl_manager = HumanInTheLoopManager()
        self.agent = None
        self.conversation_history: List[str] = []
    
    def initialize_agent(self):
        """Initialize the underlying ChatAgent"""
        self.agent = ChatAgent(
            chat_client=OpenAIChatClient(),
            instructions=self.instructions,
            tools=self.tools,
            model=self.model
        )
        print("✅ HITL Agent initialized")
    
    async def run_with_approval(self, user_message: str) -> str:
        """
        Run agent with human approval for critical operations.
        This is where the HITL magic happens!
        """
        if not self.agent:
            self.initialize_agent()
        
        print(f"\n👤 User: {user_message}")
        self.conversation_history.append(f"User: {user_message}")
        
        # Initial agent run
        response = await self.agent.run(user_message)
        
        # Check if agent wants to call any functions
        # In Microsoft Agent Framework, this is handled via the response
        # For this demo, we simulate the approval flow
        
        response_text = response.text
        self.conversation_history.append(f"Agent: {response_text}")
        
        return response_text
    
    def get_conversation_history(self) -> List[str]:
        """Get conversation history"""
        return self.conversation_history
    
    def get_approval_history(self) -> List[Dict]:
        """Get approval history"""
        return self.hitl_manager.get_approval_history()


print("✅ HITLAgent wrapper class defined!")


✅ HITLAgent wrapper class defined!


In [8]:
# Complete HITL Implementation (Functional Approach)

class BankingAgentWithHITL:
    """
    Complete banking agent with Human-in-the-Loop approval workflow.
    Demonstrates the full approval pattern.
    """
    
    def __init__(self):
        self.hitl_manager = HumanInTheLoopManager()
        self.pending_operations: List[Dict] = []
        self.agent = None
    
    def create_agent(self):
        """Create agent with all tools (safe and critical)"""
        all_tools = [
            get_account_balance,
            get_transaction_history,
            transfer_money,
            send_email,
            delete_account,
            process_payment
        ]
        
        self.agent = ChatAgent(
            chat_client=OpenAIChatClient(),
            instructions="""You are a banking assistant that helps users manage their accounts.

For read-only operations (checking balance, viewing transactions), you can execute them directly.

For critical operations (transfers, payments, deletions, emails), you MUST:
1. Explain what you plan to do
2. Show the parameters clearly
3. Wait for human approval before proceeding

Always be transparent about what actions you will take.""",
            tools=all_tools,
            model="gpt-4o-mini"
        )
        
        print("✅ Banking Agent with HITL created")
    
    async def process_request(self, user_request: str) -> str:
        """Process user request with HITL approval loop"""
        if not self.agent:
            self.create_agent()
        
        print(f"\n{'='*70}")
        print(f"👤 User Request: {user_request}")
        print(f"{'='*70}\n")
        
        # Run agent
        response = await self.agent.run(user_request)
        
        print(f"🤖 Agent Response:\n{response.text}\n")
        
        return response.text
    
    def export_logs(self):
        """Export approval logs"""
        self.hitl_manager.export_approval_log("banking_agent_approvals.json")


print("✅ BankingAgentWithHITL class defined!")


✅ BankingAgentWithHITL class defined!


In [9]:
# Initialize the HITL Banking Agent
banking_agent = BankingAgentWithHITL()
banking_agent.create_agent()

print("\n✅ HITL Banking Agent ready!")
print("\n💡 This agent will:")
print("   • Execute safe operations (read-only) automatically")
print("   • Request approval for critical operations (transfers, payments, deletions)")
print("   • Log all approval decisions")


✅ Banking Agent with HITL created

✅ HITL Banking Agent ready!

💡 This agent will:
   • Execute safe operations (read-only) automatically
   • Request approval for critical operations (transfers, payments, deletions)
   • Log all approval decisions


In [10]:
# Test safe operations (no approval required)

# Test 1: Check balance
response = await banking_agent.process_request(
    "What is the balance for account ACC001?"
)

# Test 2: View transactions
response = await banking_agent.process_request(
    "Show me the last 3 transactions for account ACC002"
)



👤 User Request: What is the balance for account ACC001?

✅ Retrieved balance for ACC001: $5000.0
🤖 Agent Response:
The balance for account ACC001 is $5000.00.


👤 User Request: Show me the last 3 transactions for account ACC002

✅ Retrieved 3 transactions for ACC002
🤖 Agent Response:
Here are the last 3 transactions for account ACC002:

1. **2025-12-10** - Online Purchase: **$-125.50**
2. **2025-12-09** - Salary Deposit: **$3000.00**
3. **2025-12-08** - Grocery Store: **$-85.20**



In [11]:
# Test critical operations (manual approval demonstration)

# Since we can't programmatically simulate the approval in the notebook,
# let's demonstrate the approval workflow manually

print("🔬 Demonstrating Manual Approval Workflow\n")
print("="*70)

# Example 1: Money Transfer
print("\n💸 SCENARIO 1: Money Transfer Request\n")

hitl = HumanInTheLoopManager()

# Simulate function call that needs approval
function_name = "transfer_money"
parameters = {
    "from_account": "ACC001",
    "to_account": "ACC002",
    "amount": 500.00
}

approved, modified = hitl.get_human_approval(
    function_name, 
    parameters,
    reasoning="User wants to transfer money to pay a bill"
)

if approved:
    result = transfer_money(
        parameters["from_account"],
        parameters["to_account"],
        parameters["amount"]
    )
    print(f"\n✅ Transfer executed:\n{result}")
else:
    print("\n❌ Transfer cancelled by human")


🔬 Demonstrating Manual Approval Workflow


💸 SCENARIO 1: Money Transfer Request


🚨 HUMAN APPROVAL REQUIRED
Function: transfer_money
Parameters:
  • from_account: ACC001
  • to_account: ACC002
  • amount: 500.0
AI Reasoning: User wants to transfer money to pay a bill

✅ Approved by human

💸 Executing transfer: $500.00 from ACC001 to ACC002

✅ Transfer executed:
{
  "status": "success",
  "transaction_id": "TXN20251217000241",
  "from_account": "ACC001",
  "to_account": "ACC002",
  "amount": 500.0,
  "timestamp": "2025-12-17T00:02:41.186086"
}


In [12]:
# Test email sending with approval

print("\n📧 SCENARIO 2: Email Sending Request\n")

hitl = HumanInTheLoopManager()

function_name = "send_email"
parameters = {
    "to_email": "customer@example.com",
    "subject": "Account Statement Ready",
    "body": "Your monthly account statement is now available."
}

approved, modified = hitl.get_human_approval(
    function_name,
    parameters,
    reasoning="Agent wants to send monthly statement notification"
)

if approved:
    result = send_email(
        parameters["to_email"],
        parameters["subject"],
        parameters["body"]
    )
    print(f"\n✅ Email sent:\n{result}")
else:
    print("\n❌ Email cancelled by human")



📧 SCENARIO 2: Email Sending Request


🚨 HUMAN APPROVAL REQUIRED
Function: send_email
Parameters:
  • to_email: customer@example.com
  • subject: Account Statement Ready
  • body: Your monthly account statement is now available.
AI Reasoning: Agent wants to send monthly statement notification

✅ Approved by human

📧 Sending email to customer@example.com
   Subject: Account Statement Ready

✅ Email sent:
{
  "status": "sent",
  "to": "customer@example.com",
  "subject": "Account Statement Ready",
  "sent_at": "2025-12-17T00:03:09.538876"
}


In [13]:
# Test payment processing with approval

print("\n💳 SCENARIO 3: Payment Processing Request\n")

hitl = HumanInTheLoopManager()

function_name = "process_payment"
parameters = {
    "amount": 299.99,
    "card_number": "****1234",
    "description": "Premium Subscription - Annual"
}

approved, modified = hitl.get_human_approval(
    function_name,
    parameters,
    reasoning="User wants to upgrade to premium subscription"
)

if approved:
    result = process_payment(
        parameters["amount"],
        parameters["card_number"],
        parameters["description"]
    )
    print(f"\n✅ Payment processed:\n{result}")
else:
    print("\n❌ Payment cancelled by human")



💳 SCENARIO 3: Payment Processing Request


🚨 HUMAN APPROVAL REQUIRED
Function: process_payment
Parameters:
  • amount: 299.99
  • card_number: ****1234
  • description: Premium Subscription - Annual
AI Reasoning: User wants to upgrade to premium subscription

✅ Approved by human

💳 Processing payment: $299.99

✅ Payment processed:
{
  "status": "completed",
  "amount": 299.99,
  "card_last4": "1234",
  "description": "Premium Subscription - Annual",
  "transaction_id": "PAY20251217000327",
  "timestamp": "2025-12-17T00:03:27.746032"
}


In [14]:
# Test account deletion with approval (most critical)

print("\n🗑️  SCENARIO 4: Account Deletion Request (HIGH RISK)\n")

hitl = HumanInTheLoopManager()

function_name = "delete_account"
parameters = {
    "account_id": "ACC003"
}

approved, modified = hitl.get_human_approval(
    function_name,
    parameters,
    reasoning="User requested to close their account permanently"
)

if approved:
    result = delete_account(parameters["account_id"])
    print(f"\n✅ Account deleted:\n{result}")
else:
    print("\n❌ Account deletion cancelled by human")



🗑️  SCENARIO 4: Account Deletion Request (HIGH RISK)


🚨 HUMAN APPROVAL REQUIRED
Function: delete_account
Parameters:
  • account_id: ACC003
AI Reasoning: User requested to close their account permanently

✅ Approved by human

🗑️  Deleting account: ACC003

✅ Account deleted:
{
  "status": "deleted",
  "account_id": "ACC003",
  "deleted_at": "2025-12-17T00:03:44.007907",
  "message": "Account ACC003 has been permanently deleted"
}


In [15]:
# View all approval decisions

print("📊 Approval History Summary\n")
print("="*70)

history = hitl.get_approval_history()

if history:
    for i, entry in enumerate(history, 1):
        print(f"\n{i}. {entry['function']}")
        print(f"   Time: {entry['timestamp']}")
        print(f"   Status: {'✅ Approved' if entry['approved'] else '❌ Rejected'}")
        if entry.get('modified'):
            print(f"   Modified: Yes")
        print(f"   Parameters: {json.dumps(entry['parameters'], indent=6)}")
else:
    print("No approval history yet")

print("\n" + "="*70)


📊 Approval History Summary


1. delete_account
   Time: 2025-12-17T00:03:44.007379
   Status: ✅ Approved
   Parameters: {
      "account_id": "ACC003"
}



In [16]:
# Export approval logs to file

hitl.export_approval_log("approval_history.json")

print("\n📄 Approval logs saved!")
print("\nLog file contains:")
print("   • Timestamp of each approval request")
print("   • Function name and parameters")
print("   • Approval decision (approved/rejected)")
print("   • Whether parameters were modified")


📄 Approval log exported to: approval_history.json

📄 Approval logs saved!

Log file contains:
   • Timestamp of each approval request
   • Function name and parameters
   • Approval decision (approved/rejected)
   • Whether parameters were modified


In [17]:
# Advanced HITL with conditional auto-approval

class SmartHITLManager(HumanInTheLoopManager):
    """
    Enhanced HITL manager with smart conditional auto-approval rules
    """
    
    def __init__(self):
        super().__init__()
        # Define auto-approval rules
        self.auto_approval_rules = {
            "transfer_money": lambda params: params.get("amount", float('inf')) < 100,
            "process_payment": lambda params: params.get("amount", float('inf')) < 50,
            "send_email": lambda params: "notification" in params.get("subject", "").lower(),
            "delete_account": lambda params: False  # Never auto-approve deletions
        }
    
    def check_auto_approval(self, function_name: str, parameters: Dict) -> bool:
        """Check if operation qualifies for auto-approval"""
        if function_name not in self.auto_approval_rules:
            return False
        
        rule = self.auto_approval_rules[function_name]
        return rule(parameters)
    
    def get_human_approval(
        self,
        function_name: str,
        parameters: Dict,
        reasoning: str = ""
    ) -> tuple[bool, Optional[str]]:
        """Enhanced approval with conditional auto-approval"""
        
        # Check auto-approval rules
        if self.check_auto_approval(function_name, parameters):
            print(f"✅ Auto-approved: {function_name} (meets safety criteria)")
            self._log_approval(function_name, parameters, approved=True)
            return True, None
        
        # Fall back to manual approval
        return super().get_human_approval(function_name, parameters, reasoning)


# Test smart HITL
print("🧠 Smart HITL Manager with Conditional Auto-Approval\n")

smart_hitl = SmartHITLManager()

# This should auto-approve (amount < 100)
print("Test 1: Small transfer (should auto-approve)")
approved, _ = smart_hitl.get_human_approval(
    "transfer_money",
    {"from_account": "ACC001", "to_account": "ACC002", "amount": 50.00}
)

print("\n" + "-"*70 + "\n")

# This requires manual approval (amount >= 100)
print("Test 2: Large transfer (requires manual approval)")
approved, _ = smart_hitl.get_human_approval(
    "transfer_money",
    {"from_account": "ACC001", "to_account": "ACC002", "amount": 5000.00}
)


🧠 Smart HITL Manager with Conditional Auto-Approval

Test 1: Small transfer (should auto-approve)
✅ Auto-approved: transfer_money (meets safety criteria)

----------------------------------------------------------------------

Test 2: Large transfer (requires manual approval)

🚨 HUMAN APPROVAL REQUIRED
Function: transfer_money
Parameters:
  • from_account: ACC001
  • to_account: ACC002
  • amount: 5000.0
✅ Approved by human



In [18]:
# Interactive HITL agent session

async def run_interactive_hitl_session():
    """
    Run an interactive HITL session where you can chat with the agent
    and approve/reject its critical actions
    """
    agent = BankingAgentWithHITL()
    agent.create_agent()
    
    print("\n" + "="*70)
    print("🤖 Interactive Banking Agent with Human-in-the-Loop")
    print("="*70)
    print("\nType 'exit' to quit\n")
    print("Try commands like:")
    print("  • Check balance for ACC001")
    print("  • Transfer $200 from ACC001 to ACC002")
    print("  • Send email about account statement")
    print("  • Show transaction history for ACC001")
    print("\n")
    
    while True:
        user_input = input("👤 You: ")
        
        if user_input.lower() in ['exit', 'quit']:
            print("\n👋 Goodbye!")
            break
        
        if not user_input.strip():
            continue
        
        response = await agent.process_request(user_input)
        print()

# Uncomment to run interactive session
# await run_interactive_hitl_session()

print("✅ Interactive HITL session function defined!")
print("\n💡 Uncomment the last line to run an interactive session")


✅ Interactive HITL session function defined!

💡 Uncomment the last line to run an interactive session
